# Group Assignment

### Import the necessary libraries

In [1]:
import numpy as np
import pandas as pd

import datetime as dt
import pandas_datareader.data as web

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from scipy.optimize import minimize
from scipy import stats

import statsmodels.api as sm

## PART I

## Examination of the January Effect
### The January Effect's main hypothesis states that stock returns are, on average, higher during January than during other months.

### 1. Download pricing data from Yahoo!Finance for each of the stocks in the ticker_list below and compute the continuously compounded daily returns using the Adjusted Closing price. Store these daily returns in a DataFrame called returns. Label the columns as 'RET_ticker', where ticker represents the trading symbol of the stock whose returns are stored in that column.

In [2]:
ticker_list = ['AAPL', 'AXP', 'BA', 'CAT', 'CSCO', 'CVX', 'DIS', 'GS', 'HD', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO',
            'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PFE', 'PG', 'TRV', 'UNH', 'UTX', 'V', 'VZ', 'WMT', 'XOM']

In [3]:
yahoo = web.DataReader(ticker_list, 'yahoo')


In [4]:
yahoo.head()

Attributes   Adj Close                                               \
Symbols           AAPL        AXP          BA        CAT       CSCO   
Date                                                                  
2015-04-06  117.195030  73.453651  131.805084  68.835617  23.425501   
2015-04-07  115.961906  72.254478  133.018997  68.033157  23.571802   
2015-04-08  115.584587  72.586555  133.927216  67.948685  23.709503   
2015-04-09  116.468048  73.020088  133.979599  68.151405  23.778347   
2015-04-10  116.964989  73.416710  134.817978  69.773262  24.131193   

Attributes                                                            ...  \
Symbols           CVX        DIS          GS          HD         IBM  ...   
Date                                                                  ...   
2015-04-06  86.980110  98.448570  177.621933  102.511871  130.132080  ...   
2015-04-07  88.297997  98.262169  177.936371  101.932228  130.156174  ...   
2015-04-08  86.768608  99.128952  178.093628  103.082649  129.979523  ...   
2015-04-09  87.004524  99.511070  180.248581  102.173004  130.372986  ...   
2015-04-10  86.971970  99.678848  180.942215  102.770508  130.790588  ...   

Attributes     Volume                                                          \
Symbols           NKE         PFE         PG        TRV        UNH        UTX   
Date                                                                            
2015-04-06  5027400.0  21480500.0  6856300.0  1052700.0  2544700.0  2857300.0   
2015-04-07  3993600.0  14331200.0  6329900.0  1363300.0  2528500.0  2269100.0   
2015-04-08  6522400.0  22974500.0  6523400.0  1544000.0  2627000.0  2484100.0   
2015-04-09  5301000.0  29283200.0  6185000.0  1217800.0  2247200.0  2088400.0   
2015-04-10  5918200.0  24353700.0  7001700.0  1699900.0  2232600.0  2254700.0   

Attributes                                                
Symbols             V          VZ        WMT         XOM  
Date                                                      
2015-04-06  8351200.0  14130000.0  6380700.0  10916400.0  
2015-04-07  6660800.0  11498700.0  6606100.0  11836100.0  
2015-04-08  6604200.0  15498600.0  6694700.0  16257100.0  
2015-04-09  5408500.0  10810900.0  3923600.0  13281800.0  
2015-04-10  4628500.0   9241900.0  5480300.0  13372400.0  

[5 rows x 168 columns]

In [5]:
adj_close = yahoo['Adj Close']
tickers = adj_close.columns

In [6]:
returns = pd.DataFrame() 
for ticker in tickers:
    returns['RET_'+ticker]= np.log(adj_close[ticker]/adj_close[ticker].shift(1))

    
returns.head() # return for each ticker 

,RET_AAPL,RET_AXP,RET_BA,RET_CAT,RET_CSCO,RET_CVX,RET_DIS,RET_GS,RET_HD,RET_IBM,...,RET_NKE,RET_PFE,RET_PG,RET_TRV,RET_UNH,RET_UTX,RET_V,RET_VZ,RET_WMT,RET_XOM
Date,,,,,,,,,,,,,,,,,,,,,
2015-04-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-04-07,-0.010578,-0.016460,0.009168,-0.011726,0.006226,0.015038,-0.001895,0.001769,-0.005670,0.000185,...,-0.001204,0.001739,-0.007859,-0.007333,-0.002978,-0.003144,0.009160,-0.007280,-0.006068,0.007257
2015-04-08,-0.003259,0.004585,0.006805,-0.001242,0.005825,-0.017473,0.008782,0.000883,0.011223,-0.001358,...,0.012272,0.002891,0.004602,0.000000,0.007133,0.002974,0.010582,0.008380,0.006562,-0.019905
2015-04-09,0.007614,0.005955,0.000391,0.002979,0.002899,0.002715,0.003847,0.012027,-0.008864,0.003023,...,-0.001190,0.011196,0.001449,-0.000373,0.003041,0.000339,-0.000903,-0.002446,-0.002347,0.006994
2015-04-10,0.004258,0.005417,0.006238,0.023519,0.014730,-0.000374,0.001685,0.003841,0.005831,0.003198,...,-0.007474,0.011636,0.005534,0.003257,0.003789,0.004908,-0.001506,0.004276,-0.002353,0.010693


### 2. Compute the monthly returns and store them in a DataFrame named returns_mon. (*Use the resample() method and the rule = 'BM' - Business Month End to compute the monthly returns*). 

In [7]:
returns_mon = returns.resample(rule='BM').sum()
returns_mon

,RET_AAPL,RET_AXP,RET_BA,RET_CAT,RET_CSCO,RET_CVX,RET_DIS,RET_GS,RET_HD,RET_IBM,...,RET_NKE,RET_PFE,RET_PG,RET_TRV,RET_UNH,RET_UTX,RET_V,RET_VZ,RET_WMT,RET_XOM
Date,,,,,,,,,,,,,,,,,,,,,
2015-04-30,-0.017426,-0.027759,-0.051596,0.072301,0.057465,0.037990,0.028833,0.022499,-0.071855,0.055515,...,-0.008965,-0.016080,-0.035430,-0.067125,-0.055012,-0.035495,0.012953,0.027415,-0.036976,0.025972
2015-05-29,0.044341,0.028888,-0.013481,-0.018119,0.016512,-0.065426,0.015063,0.051657,0.040665,0.005593,...,0.033690,0.032121,-0.014186,0.000099,0.076113,0.035091,0.040795,-0.020025,-0.043352,-0.016914
2015-06-30,-0.037938,-0.021673,-0.012892,-0.005877,-0.065198,-0.065497,0.033587,0.012531,0.002691,-0.042075,...,0.060584,-0.035739,-0.001915,-0.038947,0.019007,-0.054727,-0.022529,-0.058945,-0.046013,-0.023754
2015-07-31,-0.033481,-0.021590,0.038536,-0.066587,0.042040,-0.086455,0.055865,-0.017978,0.051729,-0.004127,...,0.064527,0.072744,-0.011725,0.093371,-0.004930,-0.100634,0.115082,0.015563,0.014695,-0.049144
2015-08-31,-0.068489,0.008640,-0.091882,-0.028247,-0.093623,-0.075930,-0.163696,-0.080282,-0.004882,-0.082799,...,-0.030579,-0.104813,-0.081881,-0.063911,-0.048090,-0.084214,-0.053478,-0.016811,-0.099358,-0.042107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-31,0.094204,0.035734,-0.116964,0.020178,0.056840,0.028448,-0.040961,0.038030,-0.003331,-0.003054,...,0.080331,0.016989,0.023001,0.007741,0.053024,0.009527,0.018206,0.019073,0.002374,0.023930
2020-01-31,0.052602,0.045769,-0.023261,-0.110192,-0.035054,-0.117596,-0.044681,0.033445,0.043548,0.069798,...,-0.050717,-0.040620,0.003679,-0.039697,-0.076057,0.002934,0.057244,-0.022089,-0.037291,-0.116279
2020-02-28,-0.121830,-0.166690,-0.139797,-0.055650,-0.140861,-0.126268,-0.161783,-0.162964,-0.046023,-0.088861,...,-0.071801,-0.108229,-0.095848,-0.094012,-0.066348,-0.135157,-0.089020,-0.093025,-0.061324,-0.174398


### 3. Create a dummy variable (labeled "JAN_DUMMY") in the DataFrame returns_mon, and assign it a value of 1 if the month is January, and 0 otherwise.

In [8]:
returns_mon.reset_index(inplace=True)

In [9]:
returns_mon.loc[returns_mon['Date'].dt.month==1, 'JAN_DUMMY']=1
returns_mon.loc[returns_mon['Date'].dt.month!=1, 'JAN_DUMMY']=0

returns_mon.head(12)

,Date,RET_AAPL,RET_AXP,RET_BA,RET_CAT,RET_CSCO,RET_CVX,RET_DIS,RET_GS,RET_HD,...,RET_PFE,RET_PG,RET_TRV,RET_UNH,RET_UTX,RET_V,RET_VZ,RET_WMT,RET_XOM,JAN_DUMMY
0,2015-04-30,-0.017426,-0.027759,-0.051596,0.072301,0.057465,0.037990,0.028833,0.022499,-0.071855,...,-0.016080,-0.035430,-0.067125,-0.055012,-0.035495,0.012953,0.027415,-0.036976,0.025972,0.0
1,2015-05-29,0.044341,0.028888,-0.013481,-0.018119,0.016512,-0.065426,0.015063,0.051657,0.040665,...,0.032121,-0.014186,0.000099,0.076113,0.035091,0.040795,-0.020025,-0.043352,-0.016914,0.0
2,2015-06-30,-0.037938,-0.021673,-0.012892,-0.005877,-0.065198,-0.065497,0.033587,0.012531,0.002691,...,-0.035739,-0.001915,-0.038947,0.019007,-0.054727,-0.022529,-0.058945,-0.046013,-0.023754,0.0
3,2015-07-31,-0.033481,-0.021590,0.038536,-0.066587,0.042040,-0.086455,0.055865,-0.017978,0.051729,...,0.072744,-0.011725,0.093371,-0.004930,-0.100634,0.115082,0.015563,0.014695,-0.049144,0.0
4,2015-08-31,-0.068489,0.008640,-0.091882,-0.028247,-0.093623,-0.075930,-0.163696,-0.080282,-0.004882,...,-0.104813,-0.081881,-0.063911,-0.048090,-0.084214,-0.053478,-0.016811,-0.099358,-0.042107,0.0
5,2015-09-30,-0.022058,-0.034342,0.002064,-0.156596,0.014196,-0.026398,0.003136,-0.081953,-0.003285,...,-0.025461,0.017811,0.006058,0.006992,-0.029016,-0.023270,-0.055868,0.001698,-0.011899,0.0
6,2015-10-30,0.080113,-0.008028,0.122870,0.121526,0.102476,0.141612,0.106984,0.076104,0.068190,...,0.073926,0.068835,0.125955,0.015142,0.100618,0.107683,0.087507,-0.124670,0.106919,0.0
7,2015-11-30,-0.005821,-0.022361,-0.011689,-0.004669,-0.057056,0.016969,-0.002377,0.016769,0.079570,...,-0.023500,-0.020368,0.014772,-0.044000,-0.017831,0.020060,-0.030978,0.027569,-0.004459,0.0
8,2015-12-31,-0.116790,-0.029608,-0.005930,-0.066734,-0.003308,-0.015005,-0.070427,-0.052896,-0.007833,...,-0.015066,0.059272,-0.009653,0.047121,0.000208,-0.018651,0.016800,0.049299,-0.046497,0.0
9,2016-01-29,-0.078223,-0.257997,-0.185328,-0.062534,-0.124718,-0.039568,-0.092250,-0.109357,-0.050319,...,-0.057049,0.036984,-0.052945,-0.021307,-0.091269,-0.040258,0.090235,0.079315,-0.001284,1.0


In [10]:
returns_mon.set_index('Date', inplace=True)
returns_mon.head(12)

,RET_AAPL,RET_AXP,RET_BA,RET_CAT,RET_CSCO,RET_CVX,RET_DIS,RET_GS,RET_HD,RET_IBM,...,RET_PFE,RET_PG,RET_TRV,RET_UNH,RET_UTX,RET_V,RET_VZ,RET_WMT,RET_XOM,JAN_DUMMY
Date,,,,,,,,,,,,,,,,,,,,,
2015-04-30,-0.017426,-0.027759,-0.051596,0.072301,0.057465,0.037990,0.028833,0.022499,-0.071855,0.055515,...,-0.016080,-0.035430,-0.067125,-0.055012,-0.035495,0.012953,0.027415,-0.036976,0.025972,0.0
2015-05-29,0.044341,0.028888,-0.013481,-0.018119,0.016512,-0.065426,0.015063,0.051657,0.040665,0.005593,...,0.032121,-0.014186,0.000099,0.076113,0.035091,0.040795,-0.020025,-0.043352,-0.016914,0.0
2015-06-30,-0.037938,-0.021673,-0.012892,-0.005877,-0.065198,-0.065497,0.033587,0.012531,0.002691,-0.042075,...,-0.035739,-0.001915,-0.038947,0.019007,-0.054727,-0.022529,-0.058945,-0.046013,-0.023754,0.0
2015-07-31,-0.033481,-0.021590,0.038536,-0.066587,0.042040,-0.086455,0.055865,-0.017978,0.051729,-0.004127,...,0.072744,-0.011725,0.093371,-0.004930,-0.100634,0.115082,0.015563,0.014695,-0.049144,0.0
2015-08-31,-0.068489,0.008640,-0.091882,-0.028247,-0.093623,-0.075930,-0.163696,-0.080282,-0.004882,-0.082799,...,-0.104813,-0.081881,-0.063911,-0.048090,-0.084214,-0.053478,-0.016811,-0.099358,-0.042107,0.0
2015-09-30,-0.022058,-0.034342,0.002064,-0.156596,0.014196,-0.026398,0.003136,-0.081953,-0.003285,-0.019942,...,-0.025461,0.017811,0.006058,0.006992,-0.029016,-0.023270,-0.055868,0.001698,-0.011899,0.0
2015-10-30,0.080113,-0.008028,0.122870,0.121526,0.102476,0.141612,0.106984,0.076104,0.068190,-0.034313,...,0.073926,0.068835,0.125955,0.015142,0.100618,0.107683,0.087507,-0.124670,0.106919,0.0
2015-11-30,-0.005821,-0.022361,-0.011689,-0.004669,-0.057056,0.016969,-0.002377,0.016769,0.079570,0.004614,...,-0.023500,-0.020368,0.014772,-0.044000,-0.017831,0.020060,-0.030978,0.027569,-0.004459,0.0
2015-12-31,-0.116790,-0.029608,-0.005930,-0.066734,-0.003308,-0.015005,-0.070427,-0.052896,-0.007833,-0.012995,...,-0.015066,0.059272,-0.009653,0.047121,0.000208,-0.018651,0.016800,0.049299,-0.046497,0.0


In [11]:
returns_mon['JAN_DUMMY'].value_counts()

0.0    56
1.0     5
Name: JAN_DUMMY, dtype: int64

### 4. Use the DataFrame returns_mon to compute the average returns in January, and during non-January months for each year and for each stock, and store them in a new DataFrame named returns_jan_mon. 

In [20]:
returns_jan_mon = returns_mon.groupby(["JAN_DUMMY", returns_mon.index.year]).mean()
returns_jan_mon.drop((0.0, 2015), axis=0, inplace=True)

In [21]:
returns_jan_mon

RET_AAPL   RET_AXP    RET_BA   RET_CAT  RET_CSCO   RET_CVX  \
JAN_DUMMY Date                                                               
0.0       2016  0.017803  0.030939  0.027287  0.038807  0.024149  0.031991   
          2017  0.031685  0.024931  0.056168  0.047322  0.022465  0.014192   
          2018 -0.004065 -0.002816 -0.006800 -0.020925  0.005882 -0.009440   
          2019  0.052970  0.018444 -0.013265  0.011367  0.003111  0.008152   
          2020 -0.077216 -0.173871 -0.314847 -0.039363 -0.048041 -0.109806   
1.0       2016 -0.078223 -0.257997 -0.185328 -0.062534 -0.124718 -0.039568   
          2017  0.046642  0.034831  0.048521  0.039264  0.024960 -0.055461   
          2018 -0.010693  0.004428  0.183671  0.037097  0.088638  0.001277   
          2019  0.053687  0.078657  0.178749  0.053226  0.095167  0.052465   
          2020  0.052602  0.045769 -0.023261 -0.110192 -0.035054 -0.117596   

                 RET_DIS    RET_GS    RET_HD   RET_IBM  ...   RET_NKE  \
JAN_DUMMY Date                                          ...             
0.0       2016  0.008995  0.037172  0.007738  0.030204  ... -0.016745   
          2017 -0.001215  0.010702  0.031193 -0.008255  ...  0.016438   
          2018  0.002246 -0.041691 -0.012183 -0.029136  ...  0.008586   
          2019  0.024738  0.015442  0.018248  0.004002  ...  0.020315   
          2020 -0.118262 -0.151719 -0.074597 -0.085598  ... -0.060932   
1.0       2016 -0.092250 -0.109357 -0.050319 -0.097864  ... -0.007871   
          2017  0.059868 -0.043226  0.025769  0.050112  ...  0.039917   
          2018  0.010732  0.050255  0.058260  0.064856  ...  0.086772   
          2019  0.016910  0.170024  0.065931  0.167670  ...  0.099300   
          2020 -0.044681  0.033445  0.043548  0.069798  ... -0.050717   

                 RET_PFE    RET_PG   RET_TRV   RET_UNH   RET_UTX     RET_V  \
JAN_DUMMY Date                                                               
0.0       2016  0.009122  0.004770  0.014277  0.031486  0.022679  0.005084   
          2017  0.015540  0.006418  0.014911  0.029318  0.015873  0.029826   
          2018  0.018119  0.008152 -0.018320  0.005836 -0.021560  0.006078   
          2019 -0.004764  0.025285  0.009984  0.009193  0.023644  0.030632   
          2020 -0.041608 -0.028581 -0.098510 -0.040845 -0.183749 -0.077766   
1.0       2016 -0.057049  0.036984 -0.052945 -0.021307 -0.091269 -0.040258   
          2017 -0.023362  0.048900 -0.038639  0.012790  0.000456  0.058375   
          2018  0.022387 -0.054600  0.100097  0.071415  0.078661  0.085761   
          2019 -0.019181  0.056194  0.047218  0.081228  0.103320  0.023002   
          2020 -0.040620  0.003679 -0.039697 -0.076057  0.002934  0.057244   

                  RET_VZ   RET_WMT   RET_XOM  
JAN_DUMMY Date                                
0.0       2016  0.008899  0.006307  0.016602  
          2017  0.010340  0.037937  0.003125  
          2018  0.006730 -0.010129 -0.018768  
          2019  0.012731  0.021387 -0.000196  
          2020 -0.024608  0.013340 -0.138744  
1.0       2016  0.090235  0.079315 -0.001284  
          2017 -0.074786 -0.035040 -0.073188  
          2018  0.032635  0.076492  0.042829  
          2019 -0.010475  0.028365  0.071990  
          2020 -0.022089 -0.037291 -0.116279  

[10 rows x 28 columns]

### 5. a. Use the DataFrame returns_mon_jan to test, for each stock, whether the average return in January is higher than the average return on non-January months. Store the t-tests and p-values in a Dataframe called results_stock, along with the average returns in January, and during non-January months for each stock. The heading for the DataFrame is found below:

In [22]:
results_stock = pd.DataFrame(
{'Ticker':[], 
 'Jan_Avg':[], 
 'NonJan_Avg':[], 
 'T_Test':[], 
 'P_Value':[]}
)

for col in returns_jan_mon.columns: # for each stock
    ttest = stats.ttest_rel(returns_jan_mon[col].xs(1, level=0), returns_jan_mon[col].xs(0, level=0))
    
    results_stock = pd.concat([results_stock,
        pd.DataFrame(   {'Ticker':[col], 
                         'Jan_Avg': returns_jan_mon[col].xs(1, level=0).mean(), 
                         'NonJan_Avg':returns_jan_mon[col].xs(0, level=0).mean(), 
                         'T_Test':[ttest[0]], 
                         'P_Value':[ttest[1]/2]})], axis=0)
    
results_stock.set_index('Ticker', inplace=True)
results_stock.sort_index(inplace=True)


In [26]:
results_stock

,Jan_Avg,NonJan_Avg,T_Test,P_Value
Ticker,,,,
RET_AAPL,0.012803,0.004235,0.237731,0.411885
RET_AXP,-0.018863,-0.020475,0.019585,0.492656
RET_BA,0.040471,-0.050291,1.007700,0.185304
RET_CAT,-0.008628,0.007442,-0.518925,0.315600
RET_CSCO,0.009799,0.001513,0.191819,0.428614
RET_CVX,-0.031777,-0.012982,-0.826381,0.227515
RET_DIS,-0.009884,-0.016699,0.219404,0.418538
RET_GS,0.020228,-0.026019,0.729766,0.252986
RET_HD,0.028638,-0.005920,1.132500,0.160355


### 5. b. Use the DataFrame results_stock to output the stocks for which the January effect hypothesis is true with a 95% degree of confidence. What about the observations where the hypothesis is true with 90% degree of confidence.

In [25]:
# pvalue that is less than or equal .05 and t greater than 0 

results_stock[(results_stock['T_Test']>0)& (results_stock['P_Value']<=.05)]

,Jan_Avg,NonJan_Avg,T_Test,P_Value
Ticker,,,,
RET_NKE,0.03348,-0.006467,2.499946,0.033385


In [27]:
#90 confidence
results_stock[(results_stock['T_Test']>0)& (results_stock['P_Value']<=.10)]

,Jan_Avg,NonJan_Avg,T_Test,P_Value
Ticker,,,,
RET_NKE,0.03348,-0.006467,2.499946,0.033385


### 6. a. Use the DataFrame returns_mon_jan to test, for each year, whether the average return in January is different from the average return on non-January months. Store the t-tests and p-values in a Dataframe called results_year, along with the average returns in January, and during non-January months for each year. The heading for the DataFrame is found below:

In [38]:
# all years
results_year = pd.DataFrame({'Year':[],
                             'Jan_Avg':[],
                             'NonJan_Avg':[],
                             'T_Test':[],
                             'P_Value':[]})
for year in returns_jan_mon.index.get_level_values('Date').unique():
    ttest=stats.ttest_rel(returns_jan_mon.xs(year,level=1).loc[1],
                          returns_jan_mon.xs(year,level=1).loc[0],
                           nan_policy='omit')
    results_year = pd.concat([results_year,
                         pd.DataFrame({
                        'Year':[year],
                        'Jan_Avg':returns_jan_mon.xs(year,level=1).loc[1].mean(),
                        'NonJan_Avg':returns_jan_mon.xs(year,level=1).loc[0].mean(),
                        'T_Test':ttest[0],
                        'P_Value':ttest[1]/2})],
                        axis = 0)
results_year.set_index('Year', inplace = True)
results_year.sort_index(inplace=True)
results_year.index=results_year.index.astype('int64')


In [39]:
results_year

,Jan_Avg,NonJan_Avg,T_Test,P_Value
Year,,,,
2016,-0.048209,0.016807,-4.128631,0.000157
2017,0.007511,0.020446,-1.642525,0.056039
2018,0.049836,-0.004736,5.578986,0.000003
2019,0.056765,0.015470,3.741390,0.000437
2020,-0.011521,-0.085398,4.193569,0.000132


### 6. b. Use the DataFrame results_year to output the observations where the January effect hypothesis is true with a 95% degree of confidence. What about the observations where the hypothesis is true with 90% degree of confidence.

In [41]:
results_year[(results_year['T_Test']>0)& (results_year['P_Value']<=.05)]

,Jan_Avg,NonJan_Avg,T_Test,P_Value
Year,,,,
2018,0.049836,-0.004736,5.578986,0.000003
2019,0.056765,0.015470,3.741390,0.000437
2020,-0.011521,-0.085398,4.193569,0.000132


In [42]:
results_year[(results_year['T_Test']>0)& (results_year['P_Value']<=.10)]

,Jan_Avg,NonJan_Avg,T_Test,P_Value
Year,,,,
2018,0.049836,-0.004736,5.578986,0.000003
2019,0.056765,0.015470,3.741390,0.000437
2020,-0.011521,-0.085398,4.193569,0.000132


### 7. Create a bar plot of the average return in January, as well as the average return across all non-January months, for each year in the sample.

### 8. Briefly explain whether the evidence is consistent with the January effect, and if not, name a couple of limitations of our dataset and/or approach.

### The following questions apply to a random sample of fifteen stocks that your group will select by running the following code 

### Random Stock Selection 

In [ ]:
np.random.seed (1000 + 1) 
ticker_list = ['AAPL', 'AXP', 'BA', 'CAT', 'CSCO', 'CVX', 'DIS', 'GS', 'HD', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO',
            'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PFE', 'PG', 'TRV', 'UNH', 'UTX', 'V', 'VZ', 'WMT', 'XOM']
stock_lst = np.random.choice(ticker_list,15,replace=False)
stock_lst

## PART II

### 1. Modify the returns_mon DataFrame to store only the monthly return for the fifteen stocks designated for your group. 

In [ ]:
returns_mon_mod = returns_mon.copy()

for col in returns_mon_mod.columns:
    ticker = col[4:]
    if ticker not in stock_lst:
        returns_mon_mod.drop(col, axis=1, inplace=True)        

In [ ]:
returns_mon_mod.head()

### 2. For each stock, compute the cumulative return series over the entire sample (no reset), and store the series in a new DataFrame named sample_ret.

In [ ]:
sample_ret = (1+returns_mon_mod[:]).cumprod()-1

In [ ]:
sample_ret.head()

### 3a. Which stocks were the top three performers. What were their cumulative returns over the entire sample period?

In [ ]:
sorted_sample=sample_ret.tail(1).iloc[:, np.argsort(sorted_sample.loc[sorted_sample.index[-1]])]
sorted_sample


In [ ]:
sorted_sample.iloc[:, [-3,-2,-1]]

### 3b. Which stocks were the worst three performers. What were their cumulative returns over the entire sample period?

In [ ]:
sorted_sample.iloc[:, [0,1,2]]

### 3c. Create a line plot of the series for the stocks with the highest three and lowest three cumulative returns.

In [ ]:
labels = sorted_sample.iloc[:, [-3,-2,-1]].columns
labels = labels.append(sorted_sample.iloc[:, [0,1,2]].columns)
labels

In [ ]:
fig = plt.figure(figsize=(16,8))
ax = fig.add_subplot(1, 1, 1)

for ticker in labels:
    ax.plot(sample_ret[ticker], label=[ticker])


ax.set_xlim(sample_ret.index[1], sample_ret.index[-1])       

ax.grid(True)

ax.set_title("Highest 3 & Lowest 3")

ax.legend(loc='best');

## PART III

### 1. Use the DataFrame returns_mon to create a new DataFrame named data_intial that only has the data from January 2013 to December 2017. In addition, create another DataFrame named data_post to store the 2018 and 2019 data.

In [ ]:
data_initial = returns_mon['01/01/2013':'12/31/2017']
data_initial.head()

In [ ]:
data_post = returns_mon['01/01/2018':'12/31/2019']
data_post.head()

### 2. Create a DataFrame named rf_rate to store the risk free rate over the period from January 2013 to December 2019. The risk-free rate  is available in the ff_factors19.csv file on Blackboard.

In [ ]:
#reading in data
rf_rate = pd.read_csv("ff_factors19.csv",
                     index_col='dateff',
                     usecols=['dateff', 'rf'],
                     parse_dates=True)

rf_rate.head()

### 3a. Use the DataFrames data_initial and rf_rate, in conjunction with Scipy' s minimize() function to find the optimal portfolio (highest Sharpe ratio) from the universe of stocks comprised of all the stocks in your sample. (Assume there are no short-selling constraints).

In [ ]:
# more finance topics lecture material


### 3b. The weights of the stocks in the optimal portfolio are:

### 3c. The optimal portfolio return is:

### 3d. The optimal portfolio standard deviation is:

### 3e. The Sharpe Ratio the optimal portfolio is:

### 4a. What is the return of the Global Minimum Variance Portfolio (GMVP)

### 4b. What is the standard deviation of the Global Minimum Variance Portfolio (GMVP)

### 4c. What are the weights of the stocks in the Global Minimum Variance Portfolio (GMVP)

### 5. Use Scipy's minimize() function to solve for the efficient frontier. In addition, plot it, add to the plot the fifteen stocks, the GMVP, the optimal portfolio, as well as the Capital Allocation Line.

## PART IV

### 1. Create a DataFrame named weights with three columns: 1) TICKER; 2) Optimal_Weights and 3) Equal_Weights for each ticker.

In [ ]:
# creating df
weights = pd.DataFrame(columns = ['TICKER', 'Optimal_Weights', 'Equal_Weights'])
weights

### 2. Using the DataFrames data_post and weights, compute the two portfolio (Optimal and Equal-weighted) monthy returns over the period January 1st, 2018 and December 31st, 2019. Store the portfolio returns into a new DataFrame named port_returns. Label the two columns as 'Optimal Portfolio' and 'EW_Portfolio'

In [ ]:
# we have to do part three before this

### 3. In a single output, provide summary statistics on the two portfolios.

In [ ]:
# one function

### 4a. Compute the two-year cumulative return series for both portfolios and plot them in the same graph. 

### 4b. What was the two-year cumulative return for the best performer? What was the two-year cumulative return for the worst performer?